# Iterabili, iteratori, generatori, `enumerate`, `range`

![iterables.png](./imgs/iterables.png)

| Concetto         | Spiegazione                                                                                                                                                                                                            |
|------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| iterable         | Un oggetto in grado di restituire i suoi membri uno alla volta, e sul quale è possibile eseguire un ciclo e che può essere trasformato in un iteratore. Esempi di iterabili sono `list`, `set`, `tuple`, `dict`, `string`, ecc.                                                |
| iterator         | Un oggetto che rappresenta uno *stream* (flusso) di dati e che può essere iterato e che implementa i metodi `__iter__` e `__next__`.                                                                                                                                                                                     |
| generator        | Un tipo speciale di funzione che non restituisce un singolo valore: restituisce un oggetto iteratore con una sequenza di valori.                                                                                       |
| lazy evaluation  | Una strategia di valutazione per cui alcuni oggetti vengono prodotti solo quando necessario. Di conseguenza, alcuni circoli di sviluppatori si riferiscono alla valutazione pigra anche come "call-by-need".           |
| iterator pattern | Un insieme di regole da seguire per implementare un iteratore in Python.                                                                                                                                               |
| `next()`         | Una funzione built-in utilizzata per restituire l'elemento successivo in un iteratore. L'oggetto deve implementare il metodo `__next__`.                                                                               |
| `iter()`         | Una funzione built-in usata per convertire un iterabile in un iteratore. L'oggetto deve implementare il metodo `__iter__`.                                                                                             |
| `for`            | Il ciclo `for` utilizza il metodo `__iter__` per ottenere un iteratore correttamente inizializzato e `__next__` per esegiuire l'iterazione. Quando rileva un errore di `StopIteration`, termina automaticamente il ciclo senza effetti collaterali. |
| `yield`          | Una keyword di Python simile alla parola chiave `return`, con la differenza che `yield` restituisce un oggetto generatore invece di un valore.                                                                         |

![iterable_iterator_generator.png](./imgs/iterable_iterator_generator.png)

## Lazy evaluation

Vedi [su Wikipedia](https://it.wikipedia.org/wiki/Valutazione_lazy).

## `<class 'generator'>` 

La forma più compatta per creare un generatore è detta [_**Generator expressions**_](https://docs.python.org/3/reference/expressions.html#generator-expressions) (*genexp*).

Una *genex* ha la forma `(<exp> for <target> in <iterable>)`.

In [ ]:
# gen = (x for x in range(3))
gen = (x for x in ['A', 'B', 'C'])

print(type(gen))

# print(gen.__next__())
# print(gen.__next__())
# print(gen.__next__())
# print(gen.__next__())  # StopIteration error

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

# for x in gen:
#     print(x)

<class 'generator'>
A
B
C


A prima vista potrebbe sembrare una "tuple comprehension", ma in questo caso le parentesi tonde non ci devono trarre in inganno.

Possiamo comunque implementare il nostro generatore, con l'uso della parola `yeld` all'interno di una funzione.

Possiamo generare una sequenza di numeri e iterarci sopra:

In [7]:
def my_generator_range(len):
    item = 0
    yield item
    item += 1
    while item < len:
        yield item
        item += 1

gen = my_generator_range(3)

print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
0
1
2


O creare un generatore che itera sopra un iterabile e modifica i suoi elementi. Il vantaggio è che esegue i calcoli solo solo quando è necessario:

In [ ]:
def my_generator_iterable(iterable):
    lenght = len(iterable)
    idx = 0
    item = iterable[0].lower()
    yield item
    idx += 1
    while idx < lenght:
        yield iterable[idx].lower()
        idx += 1


gen = my_generator_iterable(['A', 'B', 'C'])

print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
a
b
c


Naturalmente, invece di "iterare" semplicemente passo-passo un iterabile o un range, potremmo eseguire eseguire dei calcoli aggiuntivi ad ogni iterazione e restituire il risultato di quel calcolo anziché l'elemento originale.

In [1]:
def squares(start, stop):
    for i in range(start, stop):
        yield i * i

gen = squares(1, 5)
print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
4
9
16


In [2]:
gen = (i*i for i in range(1, 5))
print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
4
9
16


## `<class '*_iterator'>`

In [3]:
# iterabile = range(3)
iterabile = ['A', 'B', 'C']

iteratore = iter(iterabile)

print(type(iteratore))

print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
# print(next(iteratore))  # StopIteration error

<class 'list_iterator'>
A
B
C


## `<class 'enumerate'>`

In [4]:
enum = enumerate(['A', 'B', 'C'])
print(type(enum))

print(next(enum))
print(next(enum))
print(next(enum))
# print(next(enum))  # StopIteration error

# for idx, elem in enum:
#     print('index:', idx, 'element:', elem)

<class 'enumerate'>
(0, 'A')
(1, 'B')
(2, 'C')


## Implementare un oggetto iteratore

Per creare un oggetto iteratore, iterabile tramite la funzione built-in `next()`, è sufficiente fare una cosa così:

In [5]:
class MyNextIterator():
    def __init__(self, stop):
        self._stop = stop
        self._cursore = 0  # inizializzo il contatore del "cursore"

    def __next__(self):
        if self._cursore < self._stop:
            res = self._cursore
            self._cursore += 1
            return res
        else:
            raise StopIteration

my_obj = MyNextIterator(3)

print(type(my_obj))

print(next(my_obj))
print(next(my_obj))
print(next(my_obj))
# print(next(my_obj))  # StopIteration error

# for x in my_obj:  # TypeError: 'MyNumbers' object is not iterable
#     print(x)

<class '__main__.MyNextIterator'>
0
1
2


Tuttavia, se volessimo eseguire un ciclo `for` su questo oggetto, otterremmo un errore `TypeError: 'MyNumbers' object is not iterable` perché l'oggetto non ha implementato la il metodo `__iter__`.

> APPROFONDIMENTO: Il metodo `__iter__` viene richiamato quando è necessario ottenere un iteratore a partire da un container. Questo metodo deve restituire un nuovo oggetto iteratore che può iterare su tutti gli oggetti del contenitore. Per le mappature, deve iterare sulle chiavi del contenitore.

Per fare questo e creare un oggetto che si comporta a tutti gli effetti come un iteratore, possiamo fare così:

In [6]:
class MyFullIterator():
    def __init__(self, stop):
        self._stop = stop
        self._cursore = 0  # inizializzo il contatore del "cursore"

    def __iter__(self):
        return self        # restituisce l'oggetto creato

    def __next__(self):
        if self._cursore < self._stop:
            res = self._cursore
            self._cursore += 1
            return res
        else:
            raise StopIteration

my_iterator = MyFullIterator(3)
print(type(my_iterator))

my_init_iterator = iter(my_iterator)
print(type(my_init_iterator))

print(next(my_init_iterator))
print(next(my_init_iterator))
print(next(my_init_iterator))
# print(next(my_init_iterator))  # StopIteration error

# for x in my_iterator:
#     print(x)

<class '__main__.MyFullIterator'>
<class '__main__.MyFullIterator'>
0
1
2


## Domanda: Ma li iteratori possono essere "resettati"?

Gli iteratori di Python normalmente non possono essere "resettati": una volta esauriti, dovrebbero sollevare una`StopIteration` ogni volta che viene chiamato `next()`. Per iterare nuovamente è necessario richiedere un nuovo oggetto iteratore con la funzione `iter()`.